In [91]:
%load_ext autoreload
%autoreload 2

import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [324]:
model_name = 'gpt2'

# prompt management

In [402]:
import json

def load_json_dataset(json_path):
    with open(json_path) as file:
        dataset = json.load(file)
    return dataset

dataset = load_json_dataset('../data/antonym.json')
dataset = list(map(lambda x: tuple(x.values()), dataset))
print(f'dataset len: {len(dataset)}')


####### TODO Delete #######
dataset = dataset[:100]

dataset len: 2398


# models

In [398]:
import sys
sys.path.append('..')
torch.set_grad_enabled(False)

from src.utils.model_utils import load_gpt_model_and_tokenizer, set_seed, rsetattr, rgetattr
from src.extraction import split_activation, extract_activations, get_mean_activations
from src.utils.prompt_helper import build_prompt_txt, tokenize_from_template, tokenize_ICL
set_seed(32)

In [399]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model, config = load_gpt_model_and_tokenizer(model_name)
print(model)
print(config)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2AttentionAltered(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
          (query): WrapperModule()
          (key): WrapperModule()
          (value): WrapperModule()
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)
{'

Calculate the average of the activation on the dataset to grasp any pattern that is task-dependant and not token-dependant 

Activations have the following shape (num_layers, num_heads, seq_len, d_head), being each OV circuit output.

Still to answer:
- How this is connected to the residual stream?

Trying to use [nnsight](https://github.com/JadenFiotto-Kaufman/nnsight) and explore to know how it works

In [403]:
# select number of ICL examples
ICL_examples = 2

###### TODO: REMEMBER TO SPLIT IN TRAIN and TEST####

tok_ret, ids_ret, labels = tokenize_ICL(tokenizer, ICL_examples = ICL_examples, dataset = dataset)
print(tokenizer.decode(tok_ret[0]))
print(labels[0])
print()
print(tok_ret[0])
print(ids_ret[0])

<|endoftext|>Q:flawed
A:perfect

Q:orthodox
A:unorthodox

Q:true
A:
false

tensor([50256,    48,    25,  2704, 36825,   198,    32,    25, 25833,   628,
           48,    25, 42539,   198,    32,    25,   403, 42539,   628,    48,
           25,  7942,   198,    32,    25])
[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24]


___


In [411]:
mean_activations, outputs = get_mean_activations(
    tokenized_prompts=tok_ret,
    important_ids=ids_ret,
    model=model,
    config=config,
)
mean_activations.shape

extracting activations:   0%|          | 0/33 [00:00<?, ?it/s]

extracting activations: 100%|██████████| 33/33 [00:13<00:00,  2.44it/s]


torch.Size([12, 12, 23, 64])

compute indirect effect

In [234]:
import random

def randomize_dataset(dataset):
    # shuffle the second column (labels) and copy the original column to a third one keeping the correct label
    shuffled = list(map(lambda x: x[1], dataset))
    random.shuffle(shuffled)

    new_dataset = list(
        zip(
            list(map(lambda x: x[0], dataset)),     # input x
            shuffled,     # new shuffled label (that make no sense)
            list(map(lambda x: x[1], dataset)),     # old correct label
    ))
    
    return new_dataset

invalid
<|endoftext|>Q:daily
A:empty

Q:distribution
A:con

Q:valid
A:


In [331]:
# parameters: dataset, mean_activations, model, config, tokenizer, 
# n_shots: Number of shots in each in-context prompt
# n_trials: Number of in-context prompts to average over


def compute_indirect_effect(dataset, mean_activations, model, config, tokenizer):
    # randomize prompts to make the model unable to guess the correct answer
    randomized_dataset = randomize_dataset(dataset)

    prompts_and_infos = tokenize_ICL(
        tokenizer, 
        ICL_examples = ICL_examples, 
        dataset = randomized_dataset
    )

    # probabilities over vocab from the original model
    probs_original = torch.zeros([len(randomized_dataset), config['vocab_size']])

    for idx, (tokenized_prompt, important_ids, correct_label) in enumerate(zip(*prompts_and_infos)):
        
        # take the original result from the model (probability of correct response token y)
        with model.invoke(rand_tok_ret) as invoker:
            pass # no changes to make in the forward pass
        logits = invoker.output.logits
        logits = logits[0,-1,:] # select only the predicted token (i.e. the final token)
        # store the probabilities for each token in vocab
        probs_original[idx] = logits.softmax(dim=-1)

        # for each layer i, for each head j in the model
        for layer_i in config['n_layer']:
            for head_j in config['n_head']:
                replace_heads_w_avg(
                    layers_heads = [layer_i, head_j],
                    avg_activations = 
                    
                )
        #      substitute the head with the specific average activation
        
        # CIE(ij) = probability of correct_label token y (w/ modified model) - probability of correct_label token y (w/ original model)
        #      e.g. CIE(ij) = 0.9 - 0.1 = 0.8      head has great effect
        #      e.g. CIE(ij) = 0.3 - 0.1 = 0.2      head does not influence too much the output


    
    prob_original_model = 0
    return rand_tok_ret[0], rand_ids_ret[0]


    
rand_tok_ret, rand_ids_ret  = compute_indirect_effect(
    dataset=dataset[:200], 
    mean_activation=mean_activations,
    model=model,
    config=config,
    tokenizer=tokenizer,
)

In [344]:
def replace_heads_w_avg(layers_heads: list[tuple[int, int]], avg_activations: list[torch.tensor], important_ids: list[int], model, config):
    
    """Replace the activation of specific heads (listed in layers_heads) with the avg_activation for each specific head (listed in avg_activations). 
    Than compute the output of the model with the new activations

    Args:
        layers_heads (list[tuple[int, int]]): list of tuples each containing a layer index, head index
        avg_activations (list[torch.tensor]): list of activation for each head listed in layers_heads. The length must be the same of layers_heads
        important_ids (list[int]): list of important indexes i.e. the tokens where the average must be substituted
        model (): model
        config (): model's config


    Returns:
        torch.Tensor: logits of the model (pre-softmax) with the replaced activations
    """

    d_head = config['d_model'] // config['n_heads']

    for num_layer, num_head in layers_heads:

        with model.invoke(prompt) as invoker:
            # TODO: must substitute only certain indexes

            config['attn_hook_names'][num_layer].output[
                num_head * d_head : (num_head + 1) * d_head
            ] = avg_activations[num_layer, num_head]
        
        probs = invoker.output.logits[0,-1,:].softmax(dim=-1)


    return 1==1

torch.Size([1, 24, 50257])
torch.Size([50257])
torch.Size([50257])


In [393]:
rgetattr(model, 'transformer.h.0.attn')

GPT2AttentionAltered(
  (c_attn): Conv1D()
  (c_proj): Conv1D()
  (attn_dropout): Dropout(p=0.1, inplace=False)
  (resid_dropout): Dropout(p=0.1, inplace=False)
  (query): WrapperModule()
  (key): WrapperModule()
  (value): WrapperModule()
)

In [397]:
for layer_name in config['attn_hook_names']:
    rgetattr(model, layer_name).output

AttributeError: 'Conv1D' object has no attribute 'output'

In [384]:

getattr(
    getattr(model, config['pre-layer-module-name'])[0],
    config['post-layer-module-name']
)

AttributeError: 'GPT2LMHeadModel' object has no attribute 'transformer.h'

In [363]:
config

{'n_heads': 12,
 'n_layer': 12,
 'd_model': 768,
 'name': 'gpt2',
 'layer_name': 'transformer.h',
 'layer_hook_names': ['transformer.h.0',
  'transformer.h.1',
  'transformer.h.2',
  'transformer.h.3',
  'transformer.h.4',
  'transformer.h.5',
  'transformer.h.6',
  'transformer.h.7',
  'transformer.h.8',
  'transformer.h.9',
  'transformer.h.10',
  'transformer.h.11'],
 'attn_name': 'attn.c_proj',
 'attn_hook_names': ['transformer.h.0.attn.c_proj',
  'transformer.h.1.attn.c_proj',
  'transformer.h.2.attn.c_proj',
  'transformer.h.3.attn.c_proj',
  'transformer.h.4.attn.c_proj',
  'transformer.h.5.attn.c_proj',
  'transformer.h.6.attn.c_proj',
  'transformer.h.7.attn.c_proj',
  'transformer.h.8.attn.c_proj',
  'transformer.h.9.attn.c_proj',
  'transformer.h.10.attn.c_proj',
  'transformer.h.11.attn.c_proj']}

In [374]:
import numpy as np

d_head = config['d_model'] // config['n_heads']
n_head = 1
a = torch.Tensor(np.arange(768))
a

tensor([ 64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,
         76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
         88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
        100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110., 111.,
        112., 113., 114., 115., 116., 117., 118., 119., 120., 121., 122., 123.,
        124., 125., 126., 127.])

In [372]:
d_head

64

In [284]:
torch.zeros(size = [config['n_layer'], config['n_heads']]).shape

torch.Size([12, 12])